In [1]:
# Imports
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time


## 1/ Scraping NASA Mars news

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [3]:
# Grabbing the slides of the NASA mars news page
results = soup.find_all('div', class_='slide')
# title = results[0].find('div', class_="content_title").text.strip()
# p = results[0].find('div', class_="rollover_description_inner").text.strip()
### Why do I get only 6 paragraphs????

In [4]:
# Building a list of dictionnaries containing the title and paragraph of each article
mars_news = []
for rr in results:
    mars_news.append({'news_title':  rr.find('div', class_="content_title").text.strip(),
                     'news_p': rr.find('div', class_="rollover_description_inner").text.strip()})

## 2/ JPL Mars Space Images

In [5]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=featured&category=Mars#submit"
browser.visit(url)
time.sleep(3)

In [18]:
# Let's try to fill the research field
# browser.fill('search', 'featured')
# couldn't make it work
# >> WebDriverException: Message: unknown error: call function result missing 'value'

# Let's try to select the category
# browser.choose('category', 'Mars')
# couldn't make it work either: no error but nothing happens

# if it's not to interact with the webpage I dont really know why we would need splinter
# I guess it's for automating the scraping without having the browser open?

In [7]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")



In [11]:
# look for the url of the fullsize images
# a class="fancybox"
# data-fancybox-href
results = soup.find_all('a', class_='fancybox')
url_base = "https://www.jpl.nasa.gov"
featured_image_url = []
for rr in results:
    featured_image_url.append(url_base+rr["data-fancybox-href"])
    
featured_image_url


['https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18432_ip.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22067_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22065_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA12249_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA09023_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01887_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA08545_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA03007_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA06956_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA06893_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA06892_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA06891_hires.jpg',
 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA05572_hires.jpg',
 

In [9]:

# Close the browser after scraping
browser.quit()

## 3/ Mars Weather

## 4/ Mars Hemispheres